In [1]:
import duckdb

In [2]:
CATALOG_PATH = './catalog/**/'
con = duckdb.connect(database=':memory:')


In [4]:
con.execute("drop table if exists eurusd")
con.execute("drop table if exists eurusd_symbol")

con.execute("create table eurusd as select * from parquet_scan('{}/quote_tick/EURUSD.SIM_EIGHTCAP/*.parquet') ".format(CATALOG_PATH))
con.execute("create table eurusd_symbol as select * from parquet_scan('{}/currency_pair/EURUSD.SIM_EIGHTCAP/*.parquet') ".format(CATALOG_PATH))

In [5]:
con.execute("select * from eurusd limit 10").fetchdf()

,bid_price,ask_price,bid_size,ask_size,ts_event,ts_init
0,1157510000,1157610000,1000000000000000,1000000000000000,1534907125665000000,1534907125665000000
1,1157560000,1157660000,1000000000000000,1000000000000000,1534907145097000000,1534907145097000000
2,1157570000,1157670000,1000000000000000,1000000000000000,1534907149701000000,1534907149701000000
3,1157570000,1157680000,1000000000000000,1000000000000000,1534907154514000000,1534907154514000000
4,1157560000,1157660000,1000000000000000,1000000000000000,1534907160722000000,1534907160722000000
5,1157540000,1157650000,1000000000000000,1000000000000000,1534907165330000000,1534907165330000000
6,1157550000,1157660000,1000000000000000,1000000000000000,1534907165930000000,1534907165930000000
7,1157560000,1157660000,1000000000000000,1000000000000000,1534907168136000000,1534907168136000000
8,1157570000,1157680000,1000000000000000,1000000000000000,1534907168338000000,1534907168338000000
9,1157610000,1157710000,1000000000000000,1000000000000000,1534907168543000000,1534907168543000000


In [25]:
df = con.execute("select * from eurusd_symbol limit 1").fetch_df()
df

,id,raw_symbol,base_currency,quote_currency,price_precision,size_precision,price_increment,size_increment,lot_size,max_quantity,...,min_notional,max_price,min_price,margin_init,margin_maint,maker_fee,taker_fee,info,ts_event,ts_init
0,EURUSD.SIM_EIGHTCAP,EURUSD,EUR,USD,5,0,0.00001,1,1000,10000000,...,1_000.00 USD,None,None,0.03,0.03,0.00002,0.00002,NaN,0,0


In [26]:
con.execute("ALTER TABLE eurusd ADD COLUMN bid DOUBLE;")
con.execute("ALTER TABLE eurusd ADD COLUMN ask DOUBLE;")

con.execute("""
    UPDATE eurusd SET bid = bid_price / 10^9;
    UPDATE eurusd SET ask = ask_price / 10^9;
    """)


In [27]:
con.execute("select * from eurusd limit 10").fetchdf()

,bid_price,ask_price,bid_size,ask_size,ts_event,ts_init,bid,ask
0,1157510000,1157610000,1000000000000000,1000000000000000,1534907125665000000,1534907125665000000,1.15751,1.15761
1,1157560000,1157660000,1000000000000000,1000000000000000,1534907145097000000,1534907145097000000,1.15756,1.15766
2,1157570000,1157670000,1000000000000000,1000000000000000,1534907149701000000,1534907149701000000,1.15757,1.15767
3,1157570000,1157680000,1000000000000000,1000000000000000,1534907154514000000,1534907154514000000,1.15757,1.15768
4,1157560000,1157660000,1000000000000000,1000000000000000,1534907160722000000,1534907160722000000,1.15756,1.15766
5,1157540000,1157650000,1000000000000000,1000000000000000,1534907165330000000,1534907165330000000,1.15754,1.15765
6,1157550000,1157660000,1000000000000000,1000000000000000,1534907165930000000,1534907165930000000,1.15755,1.15766
7,1157560000,1157660000,1000000000000000,1000000000000000,1534907168136000000,1534907168136000000,1.15756,1.15766
8,1157570000,1157680000,1000000000000000,1000000000000000,1534907168338000000,1534907168338000000,1.15757,1.15768
9,1157610000,1157710000,1000000000000000,1000000000000000,1534907168543000000,1534907168543000000,1.15761,1.15771


In [28]:
con.execute("""
            drop table if exists tmp;
            create table tmp as SELECT make_timestamp(CAST(ts_event/10^3 as BIGINT)) AS ts
FROM eurusd;
select * from tmp;
            """).fetch_df()

con.execute("""
 select min(ts), max(ts) from tmp;
            """).fetch_df()

,min(ts),max(ts)
0,2018-08-22 03:05:25.665,2023-12-01 23:59:56.437


In [ ]:
# print table info
con.execute("select * from eurusd").fetchdf()

In [ ]:
# connect to questdb inside duckdb

con.execute("""
            INSTALL postgres;
LOAD postgres;

ATTACH 'user=admin password=quest host=192.168.0.36 port=9000 dbname=qdb;' AS postgres_db (TYPE POSTGRES);

            """)

In [1]:
from nautilus_trader.model.identifiers import Venue, InstrumentId
from nautilus_trader.model.data import Bar, BarType, QuoteTick
from nautilus_trader.config import BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.trading.strategy import ImportableStrategyConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos, maybe_unix_nanos_to_dt
from nautilus_trader.persistence.catalog import ParquetDataCatalog


In [2]:
CATALOG_PATH = './catalog/'
catalog = ParquetDataCatalog(CATALOG_PATH)

In [ ]:
catalog.quote_ticks(['EURUSD.SIM_EIGHTCAP'])